In [1]:
pip install mediapipe

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install chime

     |████████████████████████████████| 2.3 MB 2.3 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import cv2
import mediapipe as mp
#from collect import get_connections_list, get_distance
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
import operator
import chime
chime.theme('zelda')

def get_connections_list():
    # All landmark names and values: https://google.github.io/mediapipe/images/mobile/hand_landmarks.png
    return {
        "WRIST_TO_THUMB_MCP": (0, 2),
        "WRIST_TO_THUMB_IP": (0, 3),
        "WRIST_TO_THUMB_TIP": (0, 4),
        "WRIST_TO_INDEX_FINGER_PIP": (0, 6),
        "WRIST_TO_INDEX_FINGER_DIP": (0, 7),
        "WRIST_TO_INDEX_FINGER_TIP": (0, 8),
        "WRIST_TO_MIDDLE_FINGER_PIP": (0, 10),
        "WRIST_TO_MIDDLE_FINGER_DIP": (0, 11),
        "WRIST_TO_MIDDLE_FINGER_TIP": (0, 12),
        "WRIST_TO_RING_FINGER_PIP": (0, 14),
        "WRIST_TO_RING_FINGER_DIP": (0, 15),
        "WRIST_TO_RING_FINGER_TIP": (0, 16),
        "WRIST_TO_PINKY_PIP": (0, 18),
        "WRIST_TO_PINKY_DIP": (0, 19),
        "WRIST_TO_PINKY_TIP": (0, 20),
        "THUMB_MCP_TO_THUMB_TIP": (2, 4),
        "INDEX_FINGER_MCP_TO_INDEX_FINGER_TIP": (5, 8),
        "MIDDLE_FINGER_MCP_TO_MIDDLE_FINGER_TIP": (9, 12),
        "RING_FINGER_MCP_TO_RING_FINGER_TIP": (13, 16),
        "PINKY_MCP_TO_PINKY_TIP": (17, 20),
        "THUMB_TIP_TO_INDEX_FINGER_MCP": (4, 5),
        "THUMB_TIP_TO_INDEX_FINGER_PIP": (4, 6),
        "THUMB_TIP_TO_INDEX_FINGER_DIP": (4, 7),
        "THUMB_TIP_TO_INDEX_FINGER_TIP": (4, 8),
        "THUMB_TIP_TO_MIDDLE_FINGER_MCP": (4, 9),
        "THUMB_TIP_TO_MIDDLE_FINGER_PIP": (4, 10),
        "THUMB_TIP_TO_MIDDLE_FINGER_DIP": (4, 11),
        "THUMB_TIP_TO_MIDDLE_FINGER_TIP": (4, 12),
        "THUMB_TIP_TO_RING_FINGER_MCP": (4, 13),
        "THUMB_TIP_TO_RING_FINGER_PIP": (4, 14),
        "THUMB_TIP_TO_RING_FINGER_DIP": (4, 15),
        "THUMB_TIP_TO_RING_FINGER_TIP": (4, 16),
        "THUMB_TIP_TO_PINKY_MCP": (4, 17),
        "THUMB_TIP_TO_PINKY_PIP": (4, 18),
        "THUMB_TIP_TO_PINKY_DIP": (4, 19),
        "THUMB_TIP_TO_PINKY_TIP": (4, 20)
    }

def get_distance(first, second):
    # Calculate distance from two coordinates
    return np.sqrt(
        (first.x - second.x) ** 2 
        + (first.y - second.y) ** 2 
        + (first.z - second.z) ** 2
    )
def pred(landmarks):

# def pred(landmarks,model):
    connections_dict = get_connections_list()
    coordinates = landmarks.landmark
    data = []
    for _, values in connections_dict.items():
            data.append(get_distance(coordinates[values[0]], coordinates[values[1]])) 
    data = np.array([data])
    return data

def get_sign_list():
    # Function to get all the values in SIGN column
    df = pd.read_csv('/Users/kritika/Desktop/HGR/merged-csv-files.csv')
    return df['Sign'].unique()

def real_time_prediction():
    #model = load_model('model_a.h5')
    single_model = load_model('/Users/kritika/Desktop/HGR/final_single_hand1.h5')
    double_model = load_model('/Users/kritika/Desktop/HGR/final_double_hand_alphabetv2.h5')
    space_model = load_model('/Users/kritika/Desktop/HGR/space.h5')
    sign_list = get_sign_list()
    mp_drawing = mp.solutions.drawing_utils
    mp_hands = mp.solutions.hands
    connections_dict = get_connections_list()
    single_hand_list = ['0','1','2','3','4','5','6','7','8','9','C','I','L','O','U','V']
    double_hand_list = ['A', 'B', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'M', 'N', 'P', 'Q',
       'R', 'S', 'T', 'W', 'X', 'Y', 'Z']
    flag_check = 0
    # Initialize webcam
    # Default is zero, try changing value if it doesn't work
    fr = 1
    hands=mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)
    cap = cv2.VideoCapture(0)
    s = ""
    flow = ''
    while cap.isOpened():
        _, frame = cap.read()
        frame.flags.writeable = True
        results = hands.process(frame)
        if  results.multi_hand_landmarks:
                flag = 1  
                for res in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                            frame, res, mp_hands.HAND_CONNECTIONS)
                flow = results.multi_handedness[0].classification[0].label
                if(len(results.multi_hand_landmarks)==2):      
                    if(flag_check == 0):
                        fr = 1
                        flag_check = 1
                    if(flow=='Left'):
                        coordinates = results.multi_hand_landmarks[0].landmark
                        coordinates1 = results.multi_hand_landmarks[1].landmark
                    else:
                        coordinates = results.multi_hand_landmarks[1].landmark
                        coordinates1 = results.multi_hand_landmarks[0].landmark
                    row0 =[]
                    row1 = []
                    row2 = []
                    for _,values in connections_dict.items():
                        row0.append(get_distance(coordinates[values[0]], coordinates[values[1]]))
                        row1.append(get_distance(coordinates[values[0]], coordinates[values[1]]))
                        row0.append(get_distance(coordinates1[values[0]], coordinates1[values[1]]))
                        row2.append(get_distance(coordinates1[values[0]], coordinates1[values[1]]))
                    check_prev_ans = single_hand_list[single_model.predict(np.expand_dims(row1,axis = 0)).argmax()]
                    check_prev_ans1 = single_hand_list[single_model.predict(np.expand_dims(row2,axis = 0)).argmax()]
                    ans_check = double_model.predict(np.expand_dims(row0,axis = 0))
#                     print(row1,row0,double_hand_list[ans_check.argmax()])
                    check = ans_check.max()
                    if(check > 0.9):
                        ans = double_hand_list[ans_check.argmax()]
                        if( fr%15 == 0 and flag_check==1):
                            if(len(s) == 0):
                                s += ans
                                flag_check = 0
                            elif(s[-1]!=ans and (s[-1] == check_prev_ans or s[-1] == check_prev_ans1)):
                                s = s[:-1] + ans
                                flag_check = 0
                            elif(s[-1]!=ans):
                                s += ans
                                flag_check = 0
                            fr = 0
                    fr +=1      
                else:
                    if(flag_check == 0):
                        fr = 1
                        flag_check = 1
                    coordinates = results.multi_hand_landmarks[0].landmark
                    row0=[]
                    for _,values in connections_dict.items():
                        row0.append(get_distance(coordinates[values[0]], coordinates[values[1]])) 
                    check_ans = space_model.predict(np.expand_dims(row0,axis = 0))
                    ans = single_hand_list[single_model.predict(np.expand_dims(row0,axis = 0)).argmax()]
                    if( fr%15 == 0 and flag_check==1):
                        if(check_ans/0.5>0.85):
                            s += " "
                            chime.info()
                            flag_check = 0
                        elif(len(s) == 0):
                            s += ans
                            flag_check = 0
                        elif(s[-1]!=ans):
                            s += ans
                            flag_check = 0
                        fr = 0
                    fr +=1      
                  
        frame =cv2.flip(frame, 1) 
        cv2.putText(frame, s, (20, 90), cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 0, 0), 2)                    
        cv2.imshow('Sign Language Detection',frame)
        flag = 0
        if cv2.waitKey(20) & 0xFF == ord('c'):
            s = ""
        if cv2.waitKey(20) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
        
        

if __name__ == "__main__":
    real_time_prediction()